In [1]:
# Installing required packages
try:

  dbutils.library.installPyPI("koalas")
  dbutils.library.installPyPI("dateparser")
  dbutils.library.installPyPI("datefinder")

except:
  pass

# Importing required libraries
import dateparser, datefinder, databricks.koalas as ks, pandas as pd, numpy as np
from pyspark.sql.types import TimestampType,IntegerType,DoubleType,StringType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, sum, stddev_pop, mean, count, countDistinct, variance, when, isnan, isnull

# Initializing Variables

file_name = "dbfs:/FileStore/tables/Sample_Data.csv"
dp = lambda x: dateparser.parse(x)
dfp = lambda x: datefinder.parser.parse(x)
inf = lambda x: int(x)
fl = lambda x: float(x)
str1 = lambda x: str(x)
metrics=[]

# Creating Spark Session
def spark_session():
    
    conf = SparkConf().set('spark.driver.host', '127.0.0.1')
    session = SparkSession.builder.config(conf=conf).master("local").appName("cloud-dq-tests").getOrCreate()
    return session

# Reading Data
def read_data(session):
    
    dataframe = session.read.csv(file_name,header="true")
    return dataframe

# Inferring DataTypes of columns on a random population sample -> Return inferred datatypes for corresponding columns
def infer_types(dataframe):
  
  date_cols=[]
  dataframe = dataframe.sample(False,0.1)
  
  # Using Koalas for inferring datatypes
  ks_dataframe = ks.DataFrame(dataframe)
  
  for i in list(ks_dataframe.columns):
    
    # Checking if column has integer values
    try:
      ks_dataframe[i].apply(inf)
      date_cols.append("Integer")
      continue
      
    except:

      # Checking if column has double or floating point values
      try:
        ks_dataframe[i].apply(fl)
        date_cols.append("Double")
        continue
        
      except:
        pass

      try:
        
        # Checking if column has date or timestamp values
        try:
          # Parsing datetime values using dateparser package
          ks_dataframe[i]=ks_dataframe[i].apply(dp)
          
        except:
          # Parsing datetime values using datefinder package
          ks_dataframe[i]=ks_dataframe[i].apply(dfp)
          
        date_cols.append("Date")
        continue

      # By Default: Column has string values
      except:
        
        date_cols.append("String")
        continue
      
  return date_cols

# Converting Datatypes of columns to inferred datatypes -> Returns new dataframe with inferred datatypes
def convert_datatypes(dataframe,date_cols):
  
  for i,column in enumerate(dataframe.columns):
    
    # If inferred data type is string, convert column to String type
    if(date_cols[i]=="String"):
      dataframe = dataframe.withColumn(column, col(column).cast(StringType()))
      
    # If inferred data type is Integer, convert column to Integer type
    elif(date_cols[i]=="Integer"):
      dataframe = dataframe.withColumn(column, col(column).cast(IntegerType()))
      
    # If inferred data type is Double, convert column to Double type
    elif(date_cols[i]=="Double"):
      dataframe = dataframe.withColumn(column, col(column).cast(DoubleType()))
      
    # If inferred data type is Datetime/Timestamp, convert column to Datetime/Timestamp type
    else:
      dataframe = dataframe.withColumn(column, col(column).cast(TimestampType()))
      
  return dataframe

# Returns of sum of Numeric Column
def sum_num(dataframe,column):
  return dataframe.select(sum(column)).first()[0]

# Returns of mean of Numeric Column
def mean_num(dataframe,column):
  return dataframe.select(mean(column)).first()[0]
  
# Returns of minimum date of Timestamp Column
def minimum_date(dataframe, column):
  return dataframe.select(min(column)).first()[0].isoformat()

# Returns of minimum length of String Column
def minimum_string(dataframe,column):
  return len(dataframe.select(min(column)).first()[0])

# Returns of minimum value of Numeric Column
def minimum_num(dataframe,column):
  return dataframe.select(min(column)).first()[0]

# Returns of maximum value of Numeric Column
def maximum_num(dataframe,column):
  return dataframe.select(max(column)).first()[0]

# Returns of Maximum date of Timestamp Column
def maximum_date(dataframe, column):
  return dataframe.select(max(column)).first()[0].isoformat()

# Returns of Maximum length of String Column
def maximum_string(dataframe,column):
  return len(dataframe.select(max(column)).first()[0])

# Returns total number of records in a column
def count_records(dataframe,column):
  return dataframe.select(column).count()

# Returns distinct number of records in a column
def countdistinct(dataframe,column):
  return dataframe.select(countDistinct(column)).first()[0]

# Returns number of records having NULL / NAN values in a column
def countnull(dataframe,column):
  return dataframe.select(count(when(isnull(column) | isnan(column),column))).first()[0]
 
# Returns number of records having NULL values in a Datetime column
def countnulldate(dataframe,column):
  return dataframe.select(count(when(isnull(column),column))).first()[0]  
  
# Returns of Standard Deviation of Numeric Column
def std_dev(dataframe,column):
  return dataframe.select(stddev_pop(column)).first()[0]

# Returns of Variance of Numeric Column
def variance_num(dataframe,column):
  return dataframe.select(variance(column)).first()[0]
  
# Returns of Quantiles (5th, 25th, 50th, 75th, 95th) of Numeric Column
def quantiles(dataframe,column):
  return dataframe.approxQuantile(column,[0.05,0.25,0.5,0.75,0.95],0.0)    

# Creating Spark Session
session=spark_session()

# Reading data
df=read_data(session)

# Inferring Datatypes of Dataframe's columns
date_cols = infer_types(df)

# Converting Dataframe's columns into inferred datatypes
df = convert_datatypes(df,date_cols)

# Checking datatype of new column
for val in df.dtypes:
  
  # Adding new metrics for every column
  metric={}
  
  # If datatype is 'String'
  if(val[1]=='string'):
    
    # Computes the required metrics for a string datatype column
    record_count=count_records(df,val[0])
    distinct_record_count = countdistinct(df,val[0])
    record_null_count = countnull(df,val[0])
    completeness = round(((record_count - record_null_count)/record_count),3)
    min1=minimum_string(df,val[0])
    max1=maximum_string(df,val[0])
    
    # Storing computed metrics in a dictionary for that column
    metric['Column'] = val[0]
    metric['Number_of_Records'] = record_count
    metric['Distinct_Number_of_Records'] = distinct_record_count
    metric['Number_of_Null_Records'] = record_null_count
    metric['Completeness'] = completeness
    metric['Minimum_Length'] = min1
    metric['Maximum_Length'] = max1
    metrics.append(metric)
  
  
  # If datatype is 'TimeStamp'
  elif(val[1]=='timestamp'):
    
    # Computes the required metrics for a TimeStamp datatype column
    record_count=count_records(df,val[0])
    distinct_record_count = countdistinct(df,val[0])
    record_null_count = countnulldate(df,val[0])
    completeness = round(((record_count - record_null_count)/record_count), 3)
    min1=minimum_date(df,val[0])
    max1=maximum_date(df,val[0])
    
    # Storing computed metrics in a dictionary for that column
    metric['Column'] = val[0]
    metric['Number_of_Records'] = record_count
    metric['Distinct_Number_of_Records'] = distinct_record_count
    metric['Number_of_Null_Records'] = record_null_count
    metric['Completeness'] = completeness
    metric['Minimum_Date'] = min1
    metric['Maximum_Date'] = max1
    metrics.append(metric)
    
  
  # If datatype is 'Numeric', i.e. , Integer or Double
  elif(val[1]=='int' or val[1] == 'double'):
    
    # Computes the required metrics for a Numeric datatype column
    record_count=count_records(df,val[0])
    distinct_record_count = countdistinct(df,val[0])
    record_null_count = countnull(df,val[0])
    completeness = round(((record_count - record_null_count)/record_count) ,3)
    sum1=round(sum_num(df,val[0]),3)
    mean1=round(mean_num(df,val[0]),3)
    min1=round(minimum_num(df,val[0]),3)
    max1=round(maximum_num(df,val[0]),3)
    range1 = round(max1 - min1 , 3)
    stddev1=round(std_dev(df,val[0]),3)
    variance1=round(variance_num(df,val[0]),3)
    percentile_5, percentile_25, percentile_50, percentile_75, percentile_95 = [round(j,3) for j in quantiles(df,val[0])]
    
    # Storing computed metrics in a dictionary for that column
    metric['Column'] = val[0]
    metric['Number_of_Records'] = record_count
    metric['Distinct_Number_of_Records'] = distinct_record_count
    metric['Number_of_Null_Records'] = record_null_count
    metric['Completeness'] = completeness
    metric['Sum'] = sum1
    metric['Mean'] = mean1
    metric['Minimum'] = min1
    metric['Maximum'] = max1
    metric['Range'] = range1
    metric['Standard_Deviation'] = stddev1
    metric['Variance'] = variance1
    metric['5th_Quartile'] = percentile_5
    metric['25th_Quartile'] = percentile_25
    metric['50th_Quartile'] = percentile_50
    metric['75th_Quartile'] = percentile_75
    metric['95th_Quartile'] = percentile_95
    metric['95th_Quartile'] = percentile_95
    metrics.append(metric)
   
# Printing Computed Data Profiling Metrics
print("\nMetrics: \n")
for i in metrics:
  print(i)
  print("\n")

PyPI package koalas has been installed already. The previously-installed package is `koalas`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
PyPI package dateparser has been installed already. The previously-installed package is `dateparser`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
PyPI package datefinder has been installed already. The previously-installed package is `datefinder`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
Metrics: 

{'Column': 'Facility_ID', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 8, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 1392342, 'Mean': 53551.615, 'Minimum': 1234, 'Maximum': 98933, 'Range': 97699, 'Standard_Deviation': 31329.31, 'Variance': 1020786670.086, '5th_Quartile': 12031.0, '25th_Quartile': 32555.0, '50th_Quartile': 43355.0, '75th_Quartile': 87643.0, '95th_Quartile': 98933.0}


{'Column': 'Patient_ID', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 26, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 59968, 'Mean': 2306.462, 'Minimum': 4, 'Maximum': 7837, 'Range': 7833, 'Standard_Deviation': 2356.404, 'Variance': 5774743.778, '5th_Quartile': 5.0, '25th_Quartile': 76.0, '50th_Quartile': 748.0, '75th_Quartile': 4003.0, '95th_Quartile': 6274.0}


{'Column': 'Patient_Encounter_Number', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 26, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 1907, 'Mean': 73.346, 'Minimum': 7, 'Maximum': 678, 'Range': 671, 'Standard_Deviation': 124.017, 'Variance': 15995.435, '5th_Quartile': 8.0, '25th_Quartile': 24.0, '50th_Quartile': 50.0, '75th_Quartile': 76.0, '95th_Quartile': 96.0}


{'Column': 'Patient_Type', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 2, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Minimum_Length': 2, 'Maximum_Length': 2}


{'Column': 'Revenue_Code', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 26, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 30671, 'Mean': 1179.654, 'Minimum': 128, 'Maximum': 9011, 'Range': 8883, 'Standard_Deviation': 2278.729, 'Variance': 5400312.315, '5th_Quartile': 151.0, '25th_Quartile': 241.0, '50th_Quartile': 549.0, '75th_Quartile': 854.0, '95th_Quartile': 9003.0}


{'Column': 'CDM_ID', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 9, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 130, 'Mean': 5.0, 'Minimum': 1, 'Maximum': 10, 'Range': 9, 'Standard_Deviation': 2.774, 'Variance': 8.0, '5th_Quartile': 1.0, '25th_Quartile': 3.0, '50th_Quartile': 5.0, '75th_Quartile': 8.0, '95th_Quartile': 9.0}


{'Column': 'CDM_ID_Description', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 9, 'Number_of_Null_Records': 5, 'Completeness': 0.808, 'Minimum_Length': 26, 'Maximum_Length': 26}


{'Column': 'Total_Charge_Amount', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 23, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 1738356.35, 'Mean': 66859.86, 'Minimum': 4318.42, 'Maximum': 141630.68, 'Range': 137312.26, 'Standard_Deviation': 38878.954, 'Variance': 1572035977.207, '5th_Quartile': 17273.68, '25th_Quartile': 41297.4, '50th_Quartile': 54540.0, '75th_Quartile': 92060.02, '95th_Quartile': 140411.16}


{'Column': 'Quantity', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 14, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Sum': 308, 'Mean': 11.846, 'Minimum': 1, 'Maximum': 20, 'Range': 19, 'Standard_Deviation': 5.475, 'Variance': 31.175, '5th_Quartile': 4.0, '25th_Quartile': 8.0, '50th_Quartile': 11.0, '75th_Quartile': 17.0, '95th_Quartile': 20.0}


{'Column': 'Service_Date', 'Number_of_Records': 26, 'Distinct_Number_of_Records': 24, 'Number_of_Null_Records': 0, 'Completeness': 1.0, 'Minimum_Date': '2017-01-17T00:00:00', 'Maximum_Date': '2017-12-12T00:00:00'}


{'Column': 'Charge_Amount_Per_Quantity', 'Numb